# Raghav Gupta

July 2020

Regression Analysis using Gapminder

In [ ]:
import pandas as pd
import numpy as np
from ggplot import *
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn import *
import sklearn.datasets
import sklearn.linear_model
import matplotlib.pyplot as plt
import math as math

data = pd.read_csv("gap.tsv", sep='\t')
data.head()

In [ ]:
df_1 = data[['year', 'lifeExp']].sort_values(by=['year'])

year = df_1['year'].values
lifexp = df_1['lifeExp'].values

plt.plot(year, lifexp,'o')
plt.title("life expectancy over time")

plt.xlabel("year")
plt.ylabel("life expentancy")

plt.show()

In [ ]:
ggplot(aes(x='year', y='lifeExp'), data=data) + geom_violin() + labs(title="life expectancy over time", x = "year", y = "life expectancy")

Both the scatter plot and the violin show that life expectancy has increased over time

In [ ]:
npm = np.matrix(df_1)
X = npm[:,0] 
Y = npm[:,1]
model = LinearRegression().fit(X,Y) 
m = model.coef_[0]
c = model.intercept_
x_1 = np.linspace(year.max(), year.min())
y_1 = (x_1*m)+c

plt.plot(year,lifexp,'o')
plt.plot(x_1,y_1,'r')
plt.title('Linear Regression')
plt.xlabel("year")
plt.ylabel("lifeExp")
plt.show()

In [ ]:
result_linear = sm.ols(formula="lifeExp ~ year", data=df_1).fit()
print (result_linear.summary())

The linear trend in average life expectancy is that it increase by 0.32 years every year

Now, looking at residuals to check the robustness of our analysis. First we will look at residuals accross time, and then residuals accross continents.

In [ ]:
residual_data = df_1.copy()
residual_data['residual'] =  residual_data['lifeExp'] - (residual_data['year']*m+c)
residual = residual_data.drop(['lifeExp'], 1)

ggplot(aes(x='year', y='residual'), data = residual) + geom_violin() + labs(title="Residual over time",
         x = "year",
         y = "residual")

There is a linear trend in residuals accross years, ensuring that these residuals don't contain any information that is being left out by the analysis

In [2]:
continent_data = data[['year', 'lifeExp', 'continent']].copy()
continent_data['residual'] =  continent_data['lifeExp'] - (continent_data['year']*m+c)
clean = continent_data.drop(['lifeExp', 'year'], 1)

ggplot(aes(x='continent', y='residual'), data = clean) + geom_violin() + labs(title="residual against continent",
         x = "continent",
         y = "residuals")

NameError: name 'data' is not defined

In [ ]:
continent_list =['Africa','Americas','Asia','Europe','Oceania']
clean_cont = continent_data.drop(['lifeExp'], 1)

for c in continent_list:
    continent_tab = clean_cont.groupby(['continent']).get_group(c)
    year_plot = continent_tab['year'].values
    residual_plot = continent_tab['residual'].values
    
    z = np.polyfit(x = year_plot,y = residual_plot,deg = 1)
    f = np.poly1d(z)
    
    x_n = np.linspace(year_plot.min(), year_plot.max(), 100)
    y_n = f(x_n)
    
    plt.plot(year_plot, residual_plot,'o',x_n,y_n)
    
    plt.xlabel("year")
    plt.ylabel("residuals")
    plt.title(c)
    plt.show()

However, among continents, there is no single trend in residuals. This means that we can include an interaction term for continent and year.

In [ ]:
w = sm.ols(formula="lifeExp ~ continent*year", data=continent_data).fit()
print (w.summary())